# Arbeidsløype for grafkonversjon med Grew

*Forslag til arbeidsløype i python:*

1. Søke med mønster i korpuset (finn ut hvor mange og hvilke setninger som matcher mønsteret)
    - Stramme inn mønsteret til å bare matche riktig type setninger (ut fra hvilken NDT-til-UD-regel man skal skrive).
2. Legge til enkle transformasjoner i regelen og kjøre mot én eksempelsetning.
    - Spisse/stramme inn regelen for å redusere antallet resulterende grafer.
    - Når en regel konverterer en setning til bare én graf, og grafen ser riktig ut (ifølge UDs retningslinjer), kjør regelen på andre setninger som matchet samme mønster og sjekk at disse også er riktige.
3. Hvis flere mønstre/regler kreves for å konvertere riktig, velg en strategi som gir én riktig graf per setning.

In [9]:
from pathlib import Path

FileNotFoundError: [Errno 2] No such file or directory: 'grewpy_backend'

In [13]:
help()

Welcome to Python 3.13's help utility! If this is your first time using
Python, you should definitely check out the tutorial at
https://docs.python.org/3.13/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To get a list of available
modules, keywords, symbols, or topics, enter "modules", "keywords",
"symbols", or "topics".

Each module also comes with a one-line summary of what it does; to list
the modules whose name or summary contain a given string such as "spam",
enter "modules spam".

To quit this help utility and return to the interpreter,
enter "q", "quit" or "exit".

Help on package venv:

NAME
    venv - Virtual environment (venv) package for Python. Based on PEP 405.

MODULE REFERENCE
    https://docs.python.org/3.13/library/venv.html

    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considere

## 1. Søk etter mønstre i korpus

Stram inn mønsteret etter behov for å bare matche ønskede setninger. 

In [12]:
# Load a corpus from a file or a string

filename = "data/dev_fixed_UDfeats.conll"
corpus = grew.corpus(filename)

In [19]:
grew.corpus_size(corpus)

2409

In [13]:
(sent_ids := grew.corpus_sent_ids(corpus)[:10])

['dev_fixed_UDfeats.conll_00001',
 'dev_fixed_UDfeats.conll_00002',
 'dev_fixed_UDfeats.conll_00003',
 'dev_fixed_UDfeats.conll_00004',
 'dev_fixed_UDfeats.conll_00005',
 'dev_fixed_UDfeats.conll_00006',
 'dev_fixed_UDfeats.conll_00007',
 'dev_fixed_UDfeats.conll_00008',
 'dev_fixed_UDfeats.conll_00009',
 'dev_fixed_UDfeats.conll_00010']

In [29]:
# Hent ut en gitt setningsgraf

sent_id = sent_ids[0]
(sent_graph := grew.corpus_get(sent_id, corpus))

sent_info = [sent_graph[token] for token in sorted(sent_graph, key=int)]
words = [token[0]["form"] for token in sent_info]

print(f"""
Sentence: {sent_id}
{words}
""")


Sentence: dev_fixed_UDfeats.conll_00001
['__0__', 'Dommer', 'Finn', 'Eilertsen', 'avstår', ',', 'selvfølgelig', 'bevisst', ',', 'fra', 'å', '«', 'sette', 'ord', 'på', '»', 'det', 'inntrykk', 'retten', 'for', 'sitt', 'vedkommende', 'måtte', 'ha', 'dannet', 'seg', 'av', 'de', 'handlinger', 'retten', 'finner', 'bevist', 'og', 'av', 'lovovertrederen', '.']



In [48]:
# Søk etter mønster
testpattern = """
pattern {
    V [ upos=VERB ];
    N [ upos=NOUN ];
    e: V -[ SUBJ ]-> N; 
}
"""

# Antall matchende sentinger
n_matches = grew.corpus_count(testpattern, corpus)
print(n_matches, "sentences match the pattern in the corpus \n")

# Treff på noder og kanter
corpus_results = grew.corpus_search(testpattern, corpus)
print("5 first matches in the corpus: ", corpus_results[:5])

# setningsID fra treffene
match_ids = [match["sent_id"] for match in corpus_results]

# Se på grafen til et av treffene
(sent_graph := grew.corpus_get(match_ids[0], corpus))

sent_info = [sent_graph[token] for token in sorted(sent_graph, key=int)]
words = [token[0]["form"] for token in sent_info[1:]]

# Hent ut de matchede nodene og kantene fra setningen
nodes = grew.search(testpattern, sent_graph)

print(f"""
Sentence: {sent_id}
Text: {" ".join(words)}

Annotations: {sent_info}

Matching nodes in the selected graph:
{nodes}
""")

# Skriv grafen til en fil
grew.save(sent_graph, "graph_sample.json")

501 sentences match the pattern in the corpus 

5 first matches in the corpus:  [{'sent_id': 'dev_fixed_UDfeats.conll_02408', 'matching': {'nodes': {'V': '3', 'N': '2'}, 'edges': {'e': {'source': '3', 'label': 'SUBJ', 'target': '2'}}}}, {'sent_id': 'dev_fixed_UDfeats.conll_02406', 'matching': {'nodes': {'V': '2', 'N': '1'}, 'edges': {'e': {'source': '2', 'label': 'SUBJ', 'target': '1'}}}}, {'sent_id': 'dev_fixed_UDfeats.conll_02404', 'matching': {'nodes': {'V': '11', 'N': '10'}, 'edges': {'e': {'source': '11', 'label': 'SUBJ', 'target': '10'}}}}, {'sent_id': 'dev_fixed_UDfeats.conll_02402', 'matching': {'nodes': {'V': '6', 'N': '5'}, 'edges': {'e': {'source': '6', 'label': 'SUBJ', 'target': '5'}}}}, {'sent_id': 'dev_fixed_UDfeats.conll_02398', 'matching': {'nodes': {'V': '2', 'N': '1'}, 'edges': {'e': {'source': '2', 'label': 'SUBJ', 'target': '1'}}}}]

Sentence: dev_fixed_UDfeats.conll_00001
Text: Fire vindmøller sørger for strøm .

Annotations: [[{'form': '__0__'}, [['FINV', '3']]], 

### Visuell mønstermatching

Regelutikling anbefales å gjøres i Arborator, hvor vi har et privat prosjekt: [NDT_conversion_to_UD](https://arboratorgrew.elizia.net/#/projects/NDT_conversion_to_UD)


Mønstersøk i eksisterende UD-versjon kan også gjøres i Grew Match: http://universal.grew.fr/?corpus=UD_Norwegian-Bokmaal@2.9

## 2. Skriv en konverteringsregel med mønsteret



In [49]:
rule_name = "nominal_subj"
# regel med sammensatt mønster og kommando
test_rule = """rule nominal_subj { 
pattern {
    V [ upos=VERB ];
    N [ upos=NOUN ];
    e: V -[ SUBJ ]-> N; 
}
commands {
    del_edge e;
    add_edge V -[ nsubj ]-> N;
} }"""

# Last inn regelen som et graph rewriting system
grs_rule = grew.grs(test_rule)

# Kjør regelen på den utvalgte setningen
grew.run(grs_rule, sent_graph, rule_name)

[{'0': [{'form': '__0__'}, [['FINV', '3']]],
  '1': [{'Number': 'Plur',
    'form': 'Fire',
    'lemma': 'fire',
    'textform': 'Fire',
    'upos': 'NUM',
    'wordform': 'Fire'},
   []],
  '2': [{'Definite': 'Ind',
    'Gender': 'Fem',
    'Number': 'Plur',
    'form': 'vindmøller',
    'lemma': 'vindmølle',
    'textform': 'vindmøller',
    'upos': 'NOUN',
    'wordform': 'vindmøller'},
   [['DET', '1']]],
  '3': [{'Mood': 'Ind',
    'Tense': 'Pres',
    'VerbForm': 'Fin',
    'form': 'sørger',
    'lemma': 'sørge',
    'textform': 'sørger',
    'upos': 'VERB',
    'wordform': 'sørger'},
   [['nsubj', '2'], ['ADV', '4'], ['IP', '6']]],
  '4': [{'form': 'for',
    'lemma': 'for',
    'textform': 'for',
    'upos': 'ADP',
    'wordform': 'for'},
   [['PUTFYLL', '5']]],
  '5': [{'Definite': 'Ind',
    'Gender': 'Masc',
    'Number': 'Sing',
    'form': 'strøm',
    'lemma': 'strøm',
    'textform': 'strøm',
    'upos': 'NOUN',
    'wordform': 'strøm'},
   []],
  '6': [{'form': '.',
   

## 3. Stram inn regelen slik at den terminerer i én graf 
Hvis flere mønstre/regler kreves for å konvertere riktig, velg en strategi som gir én riktig graf per setning

## Lagre regler i filer 

Lagre reglene i GREW-format i en `grs`-fil, med valgt strategi. 


In [146]:
new_rule_file = "dummy_rule.grs"
Path(new_rule_file).write_text(rule)

238

In [157]:
strategy_file = Path("teststrategy.grs")
strategy = strategy_file.read_text()

strategy += f"""

import "{new_rule_file}"

strat {rule_name} {{ {rule_name} }}

"""

strategy_file.write_text(strategy)

223

## Konverter korpuset 


In [158]:
# fra kommandolinjen
!grew transform -grs teststrategy.grs -strat {rule_name} -i no_bokmaal-ud-train.conllu -o grew_output.conllu 

0.00% (000001)                                                                                      ^C


## Visualiser resulterende grafer

Under arbeid ....

1. Last opp conllu-filen med de(n) resulterende grafen(e): https://universaldependencies.org/conllu_viewer.html 
2. Konverter grafene fra grew-dict-format til displacy-dict-format : 

In [50]:
from typing import List

from spacy import displacy

# displacy.parse_deps()   # Lag en spacy-dependency parse av en setning


# Format the grah in displacy format
def grew_to_displacy(graphs: List[dict]) -> List[dict]:
    """Displacy format:

    .. code-block:: python

        {
            "words": [
                {"text": "This", "tag": "DT"},
                {"text": "is", "tag": "VBZ"},
                {"text": "a", "tag": "DT"},
                {"text": "sentence", "tag": "NN"}
            ],
            "arcs": [
                {"start": 0, "end": 1, "label": "nsubj", "dir": "left"},
                {"start": 2, "end": 3, "label": "det", "dir": "left"},
                {"start": 1, "end": 3, "label": "attr", "dir": "right"}
            ]
        }

    Source: https://spacy.io/usage/visualizers#manual-usage
    """
    # TODO: Implement functionality
    pass


displacy_dicts = grew_to_displacy(graphs)

displacy.render(docs=displacy_dicts, manual=True)

NameError: name 'graphs' is not defined